# Cart Pole

#### Descripcion 


+ **Espacio de observacion (Box(4,))**

+ Posicion del carro: [-2.4, 2.4]
+ Velocidad del carro: [$-\infty$, $\infty$]
+ Angulo del palo: [-41.8, 41.8]
+ Velocidad del palo en la punta: [$-\infty$, $\infty$]


+ **Espacio de accion (Discrete(2))**

+ Izquierda: 0
+ Derecha: 1


El objetivo es mantener el palo vertical moviendo a izquierda y derecha el carro.


La recompensa es +1 para cada paso temporal. El episodio se termina si el angulo es mayor a $\pm 12$ grados o si el carro sobrepasa la posicion $\pm 2.4$

Implementaremos un perceptron en torch (red neuronal) para calcular los Q-valores, en vez de usar el algoritmo Q-learning usado anteriormente.

#### Perceptron implementado en Torch

In [1]:
import torch

class Perceptron(torch.nn.Module):
    
    def __init__(self, dim_entrada, dim_salida, hardware=torch.device('cpu')):
        super(Perceptron, self).__init__()                               # herencia
        self.hardware=hardware                                           # hardware usado por el perceptron
        self.dim_entrada=dim_entrada[0]                                  # dimension de la capa de entrada
        self.dim_oculta=50                                               # dimension de la capa oculta del perceptron (50 nodos, se puede cambiar)
        self.lineal=torch.nn.Linear(self.dim_entrada, self.dim_oculta)   # entrada (transformaacion lineal)
        self.salida=torch.nn.Linear(self.dim_oculta, dim_salida)         # salida
        
    def forward(self, x):  # metodo de creacion del perceptron
        x=torch.from_numpy(np.array(x)).float().to(self.hardware)
        x=torch.nn.functional.relu(self.lineal(x)) # unidad rectificado lineal
        x=self.salida(x)
        return x

#### Clase Agente 

Se cambia la clase Agente para que los Q-valores sean obtenidos con el perceptron.

In [2]:
class Agente(object):
    
    def __init__(self, entorno):
        self.obs_dim=entorno.observation_space.shape              # dimension espacio observacion
        self.obs_sup=entorno.observation_space.high               # limite superior
        self.obs_inf=entorno.observation_space.low                # limite inferior
        self.obs_bins=NUM_BINS                                    # discretizacion de un espacio continuo, numero de bins
        self.ancho_bin=(self.obs_sup-self.obs_inf)/self.obs_bins  # ancho de cada parte de la discretizacion
        self.dim_accion=entorno.action_space.n                    # dimension espacio accion
        
        self.Q=Perceptron(self.obs_dim, self.dim_accion)                    # Q-valores
        self.Q_optimizador=torch.optim.Adam(self.Q.parameters(), lr=1e-5)   # optimizador Adam
        
        self.alfa=ALFA        # tasa de aprendizaje           
        self.gamma=GAMMA      # factor de descuento
        self.epsilon=EPSILON  # prob para escoger accion 
        
        
    def discretiza(self, obs):  # discretiza un espacio continuo
        return tuple(((obs-self.obs_inf)/self.ancho_bin).astype(int))  # binning
        
        
    def accion(self, obs):   # realiza la accion del agente
        obs_discreta=self.discretiza(obs) # discretiza espacio
        
        # politica epsilon-greedy para escoger la accion (mejor probabilidad 1-eps, peor es epsilon)
        if self.epsilon>MIN_EPSILON: self.epsilon-=DECAY_EPSILON
            
            
        if np.random.random()>self.epsilon: 
            return np.argmax(self.Q(obs_discreta).data.to(torch.device('cpu')).numpy())
        else: 
            return np.random.choice([a for a in range(self.dim_accion)])
        
        
    def aprende(self, obs, acciona, recompensa, siguiente_obs): # metodo de aprendizaje
        td_objetivo=recompensa+self.gamma*torch.max(self.Q(siguiente_obs))         # valor de aprendizaje
        td_error=torch.nn.functional.mse_loss(self.Q(obs)[acciona], td_objetivo)   # error aprendizaje
        self.Q_optimizador.zero_grad()
        td_error.backward()
        self.Q_optimizador.step()

### Ejecucion


In [3]:
# primero las constantes (se pueden considerar hiperparametros)

MIN_EPSILON=0.005                               # epsilon minimo, probabilidad minima para escoger una accion
EPSILON=1.                                      # prob para escoger accion
NUM_MAX_EPIS=50000                              # numero maximo de episodios
PASOS_POR_EPI=200                               # pasos por episodio
NUM_MAX_PASOS=NUM_MAX_EPIS*PASOS_POR_EPI        # numero maximo de pasos por episodio  
DECAY_EPSILON=500*MIN_EPSILON/NUM_MAX_PASOS     # ajuste del valor de epsilon 
ALFA=0.05                                       # tasa de aprendizaje
GAMMA=0.98                                      # factor de descuento
NUM_BINS=20                                     # numero de binnings para discretizacion

In [4]:
# se ejecuta todo

import gym
import numpy as np
import warnings
warnings.simplefilter('ignore')

if __name__=='__main__':
    
    entorno=gym.make('CartPole-v0')
    
    agente=Agente(entorno)
    primer_episodio=True
    recompensas_episodio=[]
    
    for episodio in range(NUM_MAX_EPIS):
        observacion=entorno.reset()
        recompensa_total=0.
        
        for paso in range(NUM_MAX_PASOS):
            #entorno.render()
            accion=agente.accion(observacion)
            
            siguiente_obs, recompensa, done, info=entorno.step(accion)
            agente.aprende(observacion, accion, recompensa, siguiente_obs)
            
            observacion=siguiente_obs
            recompensa_total+=recompensa
            
            if done:
                if primer_episodio:
                    max_recompensa=recompensa_total
                    primer_episodio=False
                recompensas_episodio.append(recompensa_total)
                
                if recompensa_total>max_recompensa:
                    max_recompensa=recompensa_total
                
                if episodio%2000==0:
                    print ('Episodio {} acabado en {} pasos..|..Recompensa:{}, Recompensa media:{}, Mejor Recompensa:{}'.format(episodio, paso+1, recompensa_total, np.mean(recompensas_episodio),max_recompensa))
                
                break
        entorno.close()

Episodio 0 acabado en 16 pasos..|..Recompensa:16.0, Recompensa media:16.0, Mejor Recompensa:16.0
Episodio 2000 acabado en 14 pasos..|..Recompensa:14.0, Recompensa media:22.330834582708647, Mejor Recompensa:81.0
Episodio 4000 acabado en 67 pasos..|..Recompensa:67.0, Recompensa media:22.331667083229192, Mejor Recompensa:102.0
Episodio 6000 acabado en 20 pasos..|..Recompensa:20.0, Recompensa media:22.318280286618897, Mejor Recompensa:118.0
Episodio 8000 acabado en 13 pasos..|..Recompensa:13.0, Recompensa media:22.340832395950507, Mejor Recompensa:118.0
Episodio 10000 acabado en 13 pasos..|..Recompensa:13.0, Recompensa media:22.242975702429757, Mejor Recompensa:118.0
Episodio 12000 acabado en 25 pasos..|..Recompensa:25.0, Recompensa media:22.254728772602284, Mejor Recompensa:118.0
Episodio 14000 acabado en 14 pasos..|..Recompensa:14.0, Recompensa media:22.220341404185415, Mejor Recompensa:143.0
Episodio 16000 acabado en 41 pasos..|..Recompensa:41.0, Recompensa media:22.194925317167677, Mej